In [1]:
!nvidia-smi

Mon Mar 29 17:35:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
!unzip gdrive/MyDrive/idao_data/IDAO_2021_oski.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: IDAO_2021_oski/data/track_1/idao_dataset/train/ER/-1.4537824392318726__CYGNO_60_40_ER_10_keV_930V_30cm_IDAO_iso_crop_hist_pic_run3_ev102;1.png  
  inflating: __MACOSX/IDAO_2021_oski/data/track_1/idao_dataset/train/ER/._-1.4537824392318726__CYGNO_60_40_ER_10_keV_930V_30cm_IDAO_iso_crop_hist_pic_run3_ev102;1.png  
  inflating: IDAO_2021_oski/data/track_1/idao_dataset/train/ER/-1.0829859972000122__CYGNO_60_40_ER_30_keV_930V_30cm_IDAO_iso_crop_hist_pic_run3_ev201;1.png  
  inflating: __MACOSX/IDAO_2021_oski/data/track_1/idao_dataset/train/ER/._-1.0829859972000122__CYGNO_60_40_ER_30_keV_930V_30cm_IDAO_iso_crop_hist_pic_run3_ev201;1.png  
  inflating: IDAO_2021_oski/data/track_1/idao_dataset/train/ER/1.200452208518982__CYGNO_60_40_ER_30_keV_930V_30cm_IDAO_iso_crop_hist_pic_run3_ev140;1.png  
  inflating: __MACOSX/IDAO_2021_oski/data/track_1/idao_dataset/train/ER/._1.200452208518982__CYGNO_60_40_ER_30_keV_930V_30

In [4]:
!pip install transformers
!pip install timm
!pip install albumentations==0.4.6


     |████████████████████████████████| 2.0MB 5.3MB/s 
     |████████████████████████████████| 890kB 22.6MB/s 
     |████████████████████████████████| 3.2MB 29.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=bc79d27134e5554d8f10835ce2f065bfab300c12ec1693fecd9ec3816ab3d0fd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 296kB 5.3MB/s 
     |████████████████████████████████| 122kB 4.9MB/s 
     |████████████████████████████████| 952kB 6.3MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp37-none-any.whl size=65163 sha256=a6eab6c8c6d054ebf10238bb5e71dc28323742da249a18a1b628edbf4119b2e9
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling i

In [33]:
import sys
sys.path.append("IDAO_2021_oski/src")

from collections import defaultdict
import os
import random

import numpy as np
import pandas as pd
import yaml
import shutil
from sklearn.model_selection import train_test_split, StratifiedKFold

from torch.utils.data import DataLoader
from torch import nn
import torch

from transformers import get_linear_schedule_with_warmup

from datasets import SimpleDataset
from models import Wrapper, MixUp
from pipeline_utils import training, pseudolabeling
from models import ENCODER_PARAMS



In [6]:
PATH_TO_CFG = "IDAO_2021_oski/config/config.yaml"
with open(PATH_TO_CFG, "r") as file:
    config = yaml.load(file)

DATA_ROOT = config["general"]["data_root"]

def fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

fix_seed(config["general"]["seed"])
device_ids = [str(id) for id in config["general"]["device_ids"]]
ids = ",".join(device_ids)
DEVICE = torch.device(f"cuda:{ids}")

In [7]:
DATA_ROOT = "IDAO_2021_oski/data/track_1/idao_dataset"

In [8]:
train = pd.read_csv(os.path.join(DATA_ROOT, "train.csv"), index_col=0)

val_private = pd.read_csv(os.path.join(DATA_ROOT, "val_private.csv"), index_col=0)

In [9]:
train["file_path"] = train["file_path"].str.replace("/media/paniquex/samsung_2tb/IDAO_2021_oski", "IDAO_2021_oski")
val_private["file_path"] = val_private["file_path"].str.replace("/media/paniquex/samsung_2tb/IDAO_2021_oski", "IDAO_2021_oski")

In [10]:
train.sample(10)

,0,1,file_path,target
4030,NR,20,IDAO_2021_oski/data/track_1/idao_dataset/train...,8
12593,ER,30,IDAO_2021_oski/data/track_1/idao_dataset/train...,4
7515,ER,3,IDAO_2021_oski/data/track_1/idao_dataset/train...,3
1608,NR,6,IDAO_2021_oski/data/track_1/idao_dataset/train...,11
9627,ER,30,IDAO_2021_oski/data/track_1/idao_dataset/train...,4
7460,ER,30,IDAO_2021_oski/data/track_1/idao_dataset/train...,4
8568,ER,10,IDAO_2021_oski/data/track_1/idao_dataset/train...,1
7526,ER,10,IDAO_2021_oski/data/track_1/idao_dataset/train...,1
35,NR,20,IDAO_2021_oski/data/track_1/idao_dataset/train...,8
1749,NR,1,IDAO_2021_oski/data/track_1/idao_dataset/train...,6


In [11]:
test = pd.read_csv(os.path.join(DATA_ROOT, "test.csv"), index_col=0)
test["file_path"] = test["file_path"].str.replace("/media/paniquex/samsung_2tb/IDAO_2021_oski", "IDAO_2021_oski")
test.sample(5)

,file_path,type
7498,IDAO_2021_oski/data/track_1/idao_dataset/priva...,private
4411,IDAO_2021_oski/data/track_1/idao_dataset/priva...,private
2246,IDAO_2021_oski/data/track_1/idao_dataset/priva...,private
16149,IDAO_2021_oski/data/track_1/idao_dataset/priva...,private
10109,IDAO_2021_oski/data/track_1/idao_dataset/priva...,private


In [59]:
path = "IDAO_2021_oski/data/track_1/idao_dataset" # directory with public_test and private_test directories
file_names_public = [x for x in os.listdir(os.path.join(path, "public_test")) if ".png" in x] #+ os.listdir(os.path.join(path, "private_test"))
test_csv = pd.DataFrame({"file_path": file_names_public, "type": "public"})
file_names_private = [x for x in os.listdir(os.path.join(path, "private_test")) if ".png" in x]#+ os.listdir(os.path.join(path, "private_test"))
test_csv = test_csv.append(pd.DataFrame({"file_path": file_names_private, "type": "private"})).reset_index()

test_csv.loc[test_csv["type"] == "public", "file_path"] = str(os.path.join(path, "public_test")) + "/" + test_csv["file_path"]
test_csv.loc[test_csv["type"] == "private", "file_path"] = str(os.path.join(path, "private_test")) + "/" + test_csv["file_path"]

In [60]:
TEST = test_csv

In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() 



mask_NR = (train["0"] == "NR") & ((train["1"] == 1) | (train["1"] == 6) | (train["1"] == 20))
mask_ER = (train["0"] == "ER") & ((train["1"] == 3) | (train["1"] == 10) | (train["1"] == 30))
train = train[mask_NR | mask_ER]
train.index = pd.RangeIndex(0, len(train.index))
if config["general"]["task_type"] == "regression":
    train["target"] = train["1"]
    val_private["target"] = val_private["1"]
elif config["general"]["task_type"] == "classification":
    train["target"] = le.fit_transform(train["target"])
#     val_private["target"] = le.fit_transform(val_private)
elif config["general"]["task_type"] == "joint":
    train["target_regression"] = train["1"]
    train["target_classification"] = le.fit_transform(train["0"])
    train["target"] = train["target_regression"].astype(str) + "_" + train["target_classification"].astype(str)
    
    val_private["target_regression"] = val_private["1"]
    val_private["target_classification"] = le.fit_transform(val_private["0"])


kfold = StratifiedKFold(n_splits=config["training"]["n_folds"], shuffle=True,
                        random_state=config["general"]["seed"])
for fold, (t_idx, v_idx) in enumerate(kfold.split(train, train["target"])):
    train.loc[v_idx, "kfold"] = fold


    
train.to_csv(os.path.join(DATA_ROOT, "train", "train_folds.csv"))



In [20]:
val_private

,0,1,file_path,target,target_regression,target_classification
2719,NR,30,IDAO_2021_oski/data/track_1/idao_dataset/train...,10,30,1
3783,NR,3,IDAO_2021_oski/data/track_1/idao_dataset/train...,9,3,1
4874,NR,10,IDAO_2021_oski/data/track_1/idao_dataset/train...,7,10,1
4915,NR,30,IDAO_2021_oski/data/track_1/idao_dataset/train...,10,30,1
5485,NR,3,IDAO_2021_oski/data/track_1/idao_dataset/train...,9,3,1
5781,NR,10,IDAO_2021_oski/data/track_1/idao_dataset/train...,7,10,1
7132,ER,20,IDAO_2021_oski/data/track_1/idao_dataset/train...,2,20,0
7397,ER,20,IDAO_2021_oski/data/track_1/idao_dataset/train...,2,20,0
8012,ER,1,IDAO_2021_oski/data/track_1/idao_dataset/train...,0,1,0
8142,ER,6,IDAO_2021_oski/data/track_1/idao_dataset/train...,5,6,0


In [21]:
!pip install --upgrade --force-reinstall --no-deps albumentations

     |████████████████████████████████| 81kB 4.2MB/s 
  Found existing installation: albumentations 0.4.6
    Uninstalling albumentations-0.4.6:
      Successfully uninstalled albumentations-0.4.6


In [22]:
import albumentations
from albumentations import *
from albumentations.pytorch import ToTensorV2


transforms_train = albumentations.Compose([
    ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.5),
    ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    CenterCrop(height=400,
               width=400),
    Resize(*config["preprocessing"]["img_size"]),
    Normalize(
         mean=[0.485, 0.456, 0.406],
         std=[0.229, 0.224, 0.225],
     ),
    ToTensorV2()
])

transforms_val = albumentations.Compose([
    CenterCrop(height=400,
               width=400),
    Resize(*config["preprocessing"]["img_size"]),
    Normalize(
         mean=[0.485, 0.456, 0.406],
         std=[0.229, 0.224, 0.225],
     ),
    ToTensorV2()
])

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
    
    
def focal_loss(input, target, focus=2.0, raw=False):

    if raw:
        input = torch.sigmoid(input)

    eps = 1e-7

    prob_true = input * target + (1 - input) * (1 - target)
    prob_true = torch.clamp(prob_true, eps, 1-eps)
    modulating_factor = (1.0 - prob_true).pow(focus)

    return (-modulating_factor * prob_true.log()).mean()


from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss


class AngularPenaltySMLoss(nn.Module):

    def __init__(self, in_features, out_features, loss_type="cosface", eps=1e-7, s=None, m=None):
        '''
        Angular Penalty Softmax Loss
        Three 'loss_types' available: ['arcface', 'sphereface', 'cosface']
        These losses are described in the following papers: 
        
        ArcFace: https://arxiv.org/abs/1801.07698
        SphereFace: https://arxiv.org/abs/1704.08063
        CosFace/Ad Margin: https://arxiv.org/abs/1801.05599
        '''
        super(AngularPenaltySMLoss, self).__init__()
        loss_type = loss_type.lower()
        assert loss_type in  ['arcface', 'sphereface', 'cosface']
        if loss_type == 'arcface':
            self.s = 64.0 if not s else s
            self.m = 0.5 if not m else m
        if loss_type == 'sphereface':
            self.s = 64.0 if not s else s
            self.m = 1.35 if not m else m
        if loss_type == 'cosface':
            self.s = 30.0 if not s else s
            self.m = 0.4 if not m else m
        self.loss_type = loss_type
        self.in_features = in_features
        self.out_features = out_features
        self.fc = nn.Linear(in_features, out_features, bias=False)
        self.eps = eps

    def forward(self, x, labels):
        '''
        input shape (N, in_features)
        '''
        assert len(x) == len(labels)
        assert torch.min(labels) >= 0
        assert torch.max(labels) < self.out_features
        
        for W in self.fc.parameters():
            W = F.normalize(W, p=2, dim=1)

        x = F.normalize(x, p=2, dim=1)
#         print(x.shape)
        wf = self.fc(x)
        if self.loss_type == 'cosface':
            numerator = self.s * (torch.diagonal(wf.transpose(0, 1)[labels]) - self.m)
        if self.loss_type == 'arcface':
            numerator = self.s * torch.cos(torch.acos(torch.clamp(torch.diagonal(wf.transpose(0, 1)[labels]), -1.+self.eps, 1-self.eps)) + self.m)
        if self.loss_type == 'sphereface':
            numerator = self.s * torch.cos(self.m * torch.acos(torch.clamp(torch.diagonal(wf.transpose(0, 1)[labels]), -1.+self.eps, 1-self.eps)))
        excl = torch.cat([torch.cat((wf[i, :y], wf[i, y+1:])).unsqueeze(0) for i, y in enumerate(labels)], dim=0)
        denominator = torch.exp(numerator) + torch.sum(torch.exp(self.s * excl), dim=1)
        L = numerator - torch.log(denominator)
        return -torch.mean(L), wf

In [24]:
config["training"]["loss"] = {"reg": "L1", "clf": "BCE"}

In [25]:
EPOCHS = config["training"]["n_epochs"]


criterion_aam = None
if config["general"]["task_type"] == "classification":
    if config["training"]["loss"]["clf"] == "FOCAL":
        criterion = focal_loss
    elif config["training"]["loss"]["clf"] == "AAM":
        criterion = "AAM"
        criterion_aam = AngularPenaltySMLoss
    elif config["training"]["loss"]["clf"] == "BCE":
        criterion = nn.BCELoss()
elif config["general"]["task_type"] == "regression":
    if config["training"]["loss"]["reg"] == "L1":
        criterion = nn.L1Loss()
    elif config["training"]["loss"]["reg"] == "L2":
        criterion = nn.MSELoss()
elif config["training"]["loss"] == "L1":
    criterion = nn.L1Loss()
elif config["general"]["task_type"] == "joint":
    criterion = {}
    if config["training"]["loss"]["clf"] == "FOCAL":
        criterion["clf"] = focal_loss
    elif config["training"]["loss"]["clf"] == "AAM":
        criterion["clf"] = "AAM"
        criterion_aam = AngularPenaltySMLoss
    elif config["training"]["loss"]["clf"] == "BCE":
        criterion["clf"] = nn.BCELoss()
    if config["training"]["loss"]["reg"] == "L1":
        criterion["reg"] = nn.L1Loss()
    elif config["training"]["loss"]["reg"] == "L2":
        criterion["reg"] = nn.MSELoss()

In [26]:
model_names = None
if config["training"]["finetune"]:
    model_names = [name for name in os.listdir(config['training']['models_dir']) if name.find("best_model_fold") != -1]
    model_names = sorted(model_names)

In [27]:
criterion

{'clf': BCELoss(), 'reg': L1Loss()}

In [28]:
config["general"]["out_path"] = "IDAO_2021_oski/experiments/resnest14d_1e-4_joint_BCE_L1/"
config["preprocessing"]["img_size"] = [224, 224]
config["training"]["dataloader"]["batch_size"] = 64
config["training"]["n_folds"] = 3

In [29]:
try:
    shutil.rmtree(config["general"]["out_path"])
except:
    pass

try:
    os.mkdir(config["general"]["out_path"])
except:
    pass


if config["general"]["task_type"] == "regression":
    config["general"]["classes_num"] = 1
elif config["general"]["task_type"] == "joint":
    config["general"]["classes_num"] = 2
    
    
samples2preds_all = {}
samples2trues_all = {}

models = []
for i in range(config["training"]["n_folds"]):
    model_name = config["general"]["model_name"]
    model = None
    model = ENCODER_PARAMS[model_name]["init_op"]()
    model = Wrapper(model, feat_module=None, classes_num=config["general"]["classes_num"],
                    model_name=model_name,
                    spec_augmenter=None, 
                    mixup_module=None,
                    task_type=config["general"]["task_type"],
                    activation_func=config["training"]["activation_func"],
                    criterion_aam=criterion_aam)
    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=config["training"]["lr"])
#         optimizer = Ranger(model.parameters(),
#                lr=config["training"]["lr"],
#                betas=(.90, 0.999), k=4)
    train_dataset = SimpleDataset(df=train[train["kfold"] != fold], mode="train",
                                  transform=transforms_train, classes_num=config["general"]["classes_num"],
                                  task_type=config["general"]["task_type"])

    val_dataset = SimpleDataset(df=train[train["kfold"] == fold], mode="val",
                                transform=transforms_val, classes_num=config["general"]["classes_num"],
                                task_type=config["general"]["task_type"])
    val_private_dataset = SimpleDataset(df=val_private, mode="val",
                                        transform=transforms_val, classes_num=config["general"]["classes_num"],
                                        task_type=config["general"]["task_type"])
    
    train_dataloader = DataLoader(train_dataset,
                                  **config["training"]["dataloader"])
    val_dataloader = DataLoader(val_dataset,
                                **config["validation"]["dataloader"])
    val_private_dataloader = DataLoader(val_private_dataset,
                                        **config["validation"]["dataloader"])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                      T_max=(config["training"]["n_epochs"] - config["training"]["n_epochs_flat"])  * len(train_dataloader),
                                                      eta_min=1e-8)    
    samples2preds, samples2trues, model = training(EPOCHS=EPOCHS, model=model,
                                            train_dataloader=train_dataloader, 
                                            val_dataloaders_dct={"val_dataloader": val_dataloader,
                                                                  "val_private_dataloader": val_private_dataloader},
                                            DEVICE=DEVICE, criterion=criterion,
                                            optimizer=optimizer, scheduler=scheduler,
                                            config=config, fold=i,
                                            task_type=config["general"]["task_type"], CONFIG_PATH=PATH_TO_CFG)
    models.append(model)
    samples2preds_all.update(samples2preds)
    samples2trues_all.update(samples2trues)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/gluon_resnest14-9c8fe254.pth" to /root/.cache/torch/hub/checkpoints/gluon_resnest14-9c8fe254.pth
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/186 [00:00<?, ?it/s]

Predict test without augmentations


100%|██████████| 186/186 [00:15<00:00, 11.89it/s]


EARLY STOPPING COUNTER: 1/5


  0%|          | 0/1 [00:00<?, ?it/s]

...

Predict test without augmentations


  0%|          | 0/1 [00:00<?, ?it/s]

EARLY STOPPING COUNTER: 1/5
Predict test without augmentations


100%|██████████| 1/1 [00:00<00:00,  3.29it/s]


['IDAO_2021_oski/experiments/resnest14d_1e-4_joint_BCE_L1/resnest14d_score=-257.31001', 'IDAO_2021_oski/experiments/resnest14d_1e-4_joint_BCE_L1/resnest14d_score=-56.41832', 'IDAO_2021_oski/experiments/resnest14d_1e-4_joint_BCE_L1/resnest14d_score=699.99767', 'IDAO_2021_oski/experiments/resnest14d_1e-4_joint_BCE_L1/resnest14d_score=718.70298', 'IDAO_2021_oski/experiments/resnest14d_1e-4_joint_BCE_L1/resnest14d_score=810.40058', 'IDAO_2021_oski/experiments/resnest14d_1e-4_joint_BCE_L1/resnest14d_score=872.00443', 'IDAO_2021_oski/experiments/resnest14d_1e-4_joint_BCE_L1/resnest14d_score=874.91678', 'IDAO_2021_oski/experiments/resnest14d_1e-4_joint_BCE_L1/resnest14d_score=914.59882', 'IDAO_2021_oski/experiments/resnest14d_1e-4_joint_BCE_L1/resnest14d_score=927.12513', 'IDAO_2021_oski/experiments/resnest14d_1e-4_joint_BCE_L1/resnest14d_score=934.64656']
IDAO_2021_oski/experiments/resnest14d_1e-4_joint_BCE_L1/resnest14d_score=934.64656
['IDAO_2021_oski/experiments/resnest14d_1e-4_joint_BCE_

In [61]:
import imp
#imp.reload(pipeline_utils)
imp.reload(datasets)
test = test_csv
#import datasets

In [ ]:


samples2preds_all = {}
samples2trues_all = {}
LR = config["training"]["lr"]
flag_LR = True

for j in range(config["pseudo"]["iter"]):
    with torch.no_grad():
        train, test = pipeline_utils.pseudolabeling(models, train, test, config, DEVICE, transforms_val)
        private = (train["type"].values == "private").sum()
        public = (train["type"].values == "public").sum()
        print("Pseudo labeling epoch", j)
        print("Private ratio", private / (public + private)) 
        print("Public ratio", public / (public + private))
        if flag_LR:
            LR *= config["pseudo"]["lr_coef"]
            flag_LR = False
    
    for i in range(config["training"]["n_folds"]):
        model = models[i]
        model.to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #         optimizer = Ranger(model.parameters(),
    #                lr=config["training"]["lr"],
    #                betas=(.90, 0.999), k=4)
        train_dataset = SimpleDataset(df=train[train["kfold"] != fold], mode="train",
                                      transform=transforms_train, classes_num=config["general"]["classes_num"],
                                      task_type=config["general"]["task_type"])

        val_dataset = SimpleDataset(df=train[train["kfold"] == fold], mode="val",
                                    transform=transforms_val, classes_num=config["general"]["classes_num"],
                                    task_type=config["general"]["task_type"])
        val_private_dataset = SimpleDataset(df=val_private, mode="val",
                                            transform=transforms_val, classes_num=config["general"]["classes_num"],
                                            task_type=config["general"]["task_type"])
        
        train_dataloader = DataLoader(train_dataset,
                                      **config["training"]["dataloader"])
        val_dataloader = DataLoader(val_dataset,
                                    **config["validation"]["dataloader"])
        val_private_dataloader = DataLoader(val_private_dataset,
                                            **config["validation"]["dataloader"])
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                          T_max=(config["pseudo"]["n_epochs"] - config["pseudo"]["n_epochs_flat"])  * len(train_dataloader),
                                                          eta_min=1e-8)    
        samples2preds, samples2trues, model = training(EPOCHS=config["pseudo"]["n_epochs"], model=model,
                                                train_dataloader=train_dataloader, 
                                                val_dataloaders_dct={"val_dataloader": val_dataloader,
                                                                      "val_private_dataloader": val_private_dataloader},
                                                DEVICE=DEVICE, criterion=criterion,
                                                optimizer=optimizer, scheduler=scheduler,
                                                config=config, fold=i, pseudo_iter=j+1,
                                                task_type=config["general"]["task_type"], CONFIG_PATH=PATH_TO_CFG)
        models[i] = model
        samples2preds_all.update(samples2preds)
        samples2trues_all.update(samples2trues)





  0%|          | 0/176 [00:00<?, ?it/s]


  1%|          | 1/176 [00:00<01:17,  2.25it/s]


  2%|▏         | 4/176 [00:00<00:55,  3.10it/s]


  4%|▍         | 7/176 [00:00<00:39,  4.23it/s]


  5%|▌         | 9/176 [00:00<00:30,  5.49it/s]


  6%|▋         | 11/176 [00:00<00:24,  6.81it/s]


  8%|▊         | 14/176 [00:01<00:18,  8.73it/s]


  9%|▉         | 16/176 [00:01<00:15, 10.26it/s]


 10%|█         | 18/176 [00:01<00:13, 11.97it/s]


 12%|█▏        | 21/176 [00:01<00:11, 13.86it/s]


 14%|█▎        | 24/176 [00:01<00:09, 15.71it/s]


 15%|█▌        | 27/176 [00:01<00:09, 15.53it/s]


 16%|█▋        | 29/176 [00:01<00:08, 16.57it/s]


 18%|█▊        | 31/176 [00:01<00:09, 15.95it/s]


 19%|█▉        | 34/176 [00:02<00:08, 17.25it/s]


 21%|██        | 37/176 [00:02<00:07, 17.88it/s]


 23%|██▎       | 40/176 [00:02<00:07, 18.68it/s]


 24%|██▍       | 42/176 [00:02<00:07, 16.81it/s]


 26%|██▌       | 45/176 [00:02<00:07, 17.75it/s]


 27%|██▋       | 48/176 [00:02<00:06, 19.

Pseudo labeling epoch 0
Private ratio 0.8976632302405498
Public ratio 0.10233676975945018





1.1273:   0%|          | 0/366 [00:03<?, ?it/s]


1.1273:   0%|          | 1/366 [00:03<20:48,  3.42s/it]


1.0721:   0%|          | 1/366 [00:03<20:48,  3.42s/it]


1.0721:   1%|          | 2/366 [00:03<15:05,  2.49s/it]


1.0227:   1%|          | 2/366 [00:04<15:05,  2.49s/it]


1.0227:   1%|          | 3/366 [00:04<11:14,  1.86s/it]


0.9611:   1%|          | 3/366 [00:04<11:14,  1.86s/it]


0.9611:   1%|          | 4/366 [00:04<08:28,  1.40s/it]


0.9605:   1%|          | 4/366 [00:04<08:28,  1.40s/it]


0.9605:   1%|▏         | 5/366 [00:04<06:34,  1.09s/it]


0.9279:   1%|▏         | 5/366 [00:05<06:34,  1.09s/it]


0.9279:   2%|▏         | 6/366 [00:05<05:15,  1.14it/s]


0.8707:   2%|▏         | 6/366 [00:05<05:15,  1.14it/s]


0.8707:   2%|▏         | 7/366 [00:05<04:22,  1.37it/s]


0.8347:   2%|▏         | 7/366 [00:05<04:22,  1.37it/s]


0.8347:   2%|▏         | 8/366 [00:05<03:41,  1.62it/s]


0.8496:   2%|▏         | 8/366 [00:06<03:41,  1.62it/s]


0.8496:   2%|▏     

Predict test without augmentations





  1%|          | 1/186 [00:00<01:35,  1.95it/s]


  2%|▏         | 3/186 [00:00<01:09,  2.64it/s]


  3%|▎         | 5/186 [00:00<00:52,  3.47it/s]


  4%|▍         | 7/186 [00:00<00:39,  4.52it/s]


  5%|▍         | 9/186 [00:01<00:31,  5.54it/s]


  6%|▌         | 11/186 [00:01<00:26,  6.70it/s]


  7%|▋         | 13/186 [00:01<00:21,  8.15it/s]


  8%|▊         | 15/186 [00:01<00:19,  8.81it/s]


  9%|▉         | 17/186 [00:01<00:17,  9.61it/s]


 10%|█         | 19/186 [00:01<00:14, 11.23it/s]


 11%|█▏        | 21/186 [00:02<00:17,  9.70it/s]


 12%|█▏        | 23/186 [00:02<00:14, 11.07it/s]


 13%|█▎        | 25/186 [00:02<00:15, 10.29it/s]


 15%|█▍        | 27/186 [00:02<00:13, 11.81it/s]


 16%|█▌        | 29/186 [00:02<00:14, 11.10it/s]


 17%|█▋        | 31/186 [00:02<00:12, 12.25it/s]


 18%|█▊        | 33/186 [00:03<00:13, 11.38it/s]


 19%|█▉        | 35/186 [00:03<00:11, 12.95it/s]


 20%|█▉        | 37/186 [00:03<00:12, 12.05it/s]


 21%|██        | 39/186 [00:03<00

...




100%|██████████| 1/1 [00:00<00:00,  3.01it/s]



  0%|          | 0/366 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



0.3901:   0%|          | 0/366 [00:03<?, ?it/s]


0.3901:   0%|          | 1/366 [00:03<18:39,  3.07s/it]


0.4677:   0%|          | 1/366 [00:03<18:39,  3.07s/it]


0.4677:   1%|          | 2/366 [00:03<13:39,  2.25s/it]


0.4765:   1%|          | 2/366 [00:03<13:39,  2.25s/it]


0.4765:   1%|          | 3/366 [00:03<10:15,  1.70s/it]


0.5492:   1%|          | 3/366 [00:04<10:15,  1.70s/it]


0.5492:   1%|          | 4/366 [00:04<07:51,  1.30s/

Predict test without augmentations





  1%|          | 1/186 [00:00<01:29,  2.06it/s]


  2%|▏         | 3/186 [00:00<01:05,  2.80it/s]


  3%|▎         | 5/186 [00:00<00:50,  3.58it/s]


  4%|▍         | 7/186 [00:00<00:38,  4.61it/s]


  5%|▍         | 9/186 [00:01<00:31,  5.61it/s]


  6%|▌         | 11/186 [00:01<00:25,  6.98it/s]


  7%|▋         | 13/186 [00:01<00:21,  7.93it/s]


  8%|▊         | 15/186 [00:01<00:19,  8.56it/s]


  9%|▉         | 17/186 [00:01<00:17,  9.90it/s]


 10%|█         | 19/186 [00:01<00:16,  9.98it/s]


 11%|█▏        | 21/186 [00:02<00:14, 11.26it/s]


 12%|█▏        | 23/186 [00:02<00:15, 10.27it/s]


 13%|█▎        | 25/186 [00:02<00:13, 11.77it/s]


 15%|█▍        | 27/186 [00:02<00:13, 12.04it/s]


 16%|█▌        | 29/186 [00:02<00:12, 12.44it/s]


 17%|█▋        | 31/186 [00:02<00:13, 11.73it/s]


 18%|█▊        | 33/186 [00:03<00:12, 12.00it/s]


 19%|█▉        | 35/186 [00:03<00:11, 13.23it/s]


 20%|█▉        | 37/186 [00:03<00:12, 11.83it/s]


 21%|██        | 39/186 [00:03<00

EARLY STOPPING COUNTER: 1/5
Predict test without augmentations





100%|██████████| 1/1 [00:00<00:00,  2.85it/s]



  0%|          | 0/366 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



0.6257:   0%|          | 0/366 [00:03<?, ?it/s]


0.6257:   0%|          | 1/366 [00:03<21:25,  3.52s/it]


0.5749:   0%|          | 1/366 [00:03<21:25,  3.52s/it]


0.5749:   1%|          | 2/366 [00:03<15:42,  2.59s/it]


0.6069:   1%|          | 2/366 [00:04<15:42,  2.59s/it]


0.6069:   1%|          | 3/366 [00:04<11:39,  1.93s/it]


0.6739:   1%|          | 3/366 [00:04<11:39,  1.93s/it]


0.6739:   1%|          | 4/366 [00:04<08:52,  1.47s/